### Data Scraping

Sentiment analysis on post-match threads from r/gunners

In [22]:
import pandas as pd
import configparser
import praw

config = configparser.ConfigParser()
config.read(r"C:\Users\Timothy Lim\Documents\credentials.ini")

credentials = config['reddit credentials']

# login
reddit = praw.Reddit(
    client_id=credentials['client_id'],
    client_secret=credentials['client_secret'],
    username=credentials['username'],
    password=credentials['password'],
    user_agent=f"testscript by u/{credentials['username']}"
)


# print(reddit.user.me())

whatsGoingOn_brotha


In [48]:
# obtain the relevant posts
gunners = reddit.subreddit('gunners')
postMatch = gunners.search(query='flair_name:"Post-Match Thread"', sort='new')

In [61]:
# get the top 100 comments for each post
from progressbar import ProgressBar

comments = {}
n_comments = 100
pbar = ProgressBar()

for post in pbar(list(postMatch)):
    comments[post.title] = post.comments[:n_comments]

100% |########################################################################|
